The requirements are:
google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread oauth2client, re, numpy

Please do:

pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread oauth2client, numpy

to install them

In [3]:
from lib.youtube_handler import *
from lib.gsheet_manager import *
import time
import random


client_secrets_file = "my_client.json"  # Path to your client secrets file
youtube_manager = YouTubeManager(client_secrets_file)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=195848401684-rshqm3h42ijd18qvk9n619snq9nvo7tg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53005%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=ns6AwRHAALHoI73oPREIP7cxQXVq77&access_type=offline


In [4]:
from lib.gsheet_manager import *
service_account_file = "sheet_client.json"
playlist_id = "PLP641Bf8QxbVdSd3qQGSePIx1HChpmfmQ"  # Replace with your actual playlist ID

auth_users = SheetAuthorizedUsersManager(service_account_file).get_usernames()
debug = SheetDebugManager(service_account_file)
requests = SheetRequestManager(service_account_file)
queue = SheetQueueManager(service_account_file)


In [12]:
youtube_manager.remove_all_videos_from_playlist( playlist_id)

All videos have been removed from the playlist.


In [10]:
youtude_interplay = ["https://www.youtube.com/watch?v=owu16UvLqGo",
                     "https://www.youtube.com/watch?v=n8IiMSXqIAo",
                     "https://www.youtube.com/watch?v=lNgH9b4xq0c" ]

while True:
    records = requests.read_records();
    if len(records)>0:
        for record in requests.read_records():
            #Validate user
            current_user = str(record['Canta']).lower()
            if current_user in auth_users: #We validate proper user
                current_link = record['Link de Youtube']
                if youtube_manager.is_valid_youtube_link(current_link):
                    send_youtube =True                        
                    try:
                        queue.add_record(list(record.values()))
                    except:
                        debug.add_record(list(record.values()))
                        send_youtube =False                        
                    if send_youtube:
                        choice= random.randint(0,2)
                        youtube_manager.add_video_to_playlist(youtude_interplay[choice], playlist_id)
                        youtube_manager.add_video_to_playlist(current_link, playlist_id)                    
                else:
                    debug.add_record(list(record.values()))            
            else:
                debug.add_record(list(record.values()))
        requests.delete_all_records()
    time.sleep(10) # Sleep for 3 seconds


True owu16UvLqGo
Added video to playlist: {'kind': 'youtube#playlistItem', 'etag': '2HlaW6QYfWLp-RxZ2Inty_v5fEk', 'id': 'UExQNjQxQmY4UXhiVmRTZDNxUUdTZVBJeDFIQ2hwbWZtUS5ENjI1QUI0MDI5NEQzODFE', 'snippet': {'publishedAt': '2024-02-02T18:31:14Z', 'channelId': 'UCPUzd2HrXUPQGjt3VYvhwLw', 'title': 'kai interplay2', 'description': '', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/owu16UvLqGo/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/owu16UvLqGo/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/owu16UvLqGo/hqdefault.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.com/vi/owu16UvLqGo/sddefault.jpg', 'width': 640, 'height': 480}, 'maxres': {'url': 'https://i.ytimg.com/vi/owu16UvLqGo/maxresdefault.jpg', 'width': 1280, 'height': 720}}, 'channelTitle': 'JoseH', 'playlistId': 'PLP641Bf8QxbVdSd3qQGSePIx1HChpmfmQ', 'position': 0, 'resourceId': {'kind': 'youtube#video', 'videoId': 'owu16Uv